주의점
1. 문장이 끊겨서 나눠져있음 (일상대화 데이터)<br>
`<SEP>` 토큰 관련

2. 라벨과 토큰 분리 관련<br>
봉구스 밥버거라는 --> '봉구스 밥버거' 로 라벨링되어있음

3. 비식별화 된 데이터 처리
%company name, name etc


오류


"id": "SDRW2000000179.1.1.87",
"form": "콜라겐을 챙겨 먹으려고 | 하는데"

| -> 토큰 개수 때문에 제거


"id": "SDRW2000000231.1.1.342",
"form": "연기를 조금 해보고 싶어 ."

. -> 토큰 취급 (띄어쓰기)


In [1]:
from transformers import BertModel, DistilBertModel
bert_model = BertModel.from_pretrained('monologg/kobert')

c:\Projects\AISec\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from tokenization_kobert import KoBertTokenizer
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert') # monologg/distilkobert도 동일
tokenizer.tokenize("[CLS] 한국어 모델을 공유합니다. [SEP]")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


['[CLS]', '▁한국', '어', '▁모델', '을', '▁공유', '합니다', '.', '[SEP]']

In [10]:
print(tokenizer.tokenize("봉구스 밥버거라는 곳에서 근무를 했었을 때에도. [SEP]"))

['▁봉', '구', '스', '▁밥', '버', '거', '라는', '▁곳', '에서', '▁근무', '를', '▁했', '었', '을', '▁때', '에도', '.', '[SEP]']


In [5]:
import json

with open('./NIKL_NE_2022_v1.0/SXNE2202211218.json', 'r', encoding='utf-8') as f:
    test = json.load(f)

In [36]:
test['document'][0]['sentence'][5:11]

[{'id': 'SDRW2000000006.1.1.6',
  'form': '그래서 떡볶이',
  'word': [{'id': 1, 'form': '그래서', 'begin': 0, 'end': 3},
   {'id': 2, 'form': '떡볶이', 'begin': 4, 'end': 7}],
  'NE': [{'id': 1, 'form': '떡볶이', 'label': 'CV_FOOD', 'begin': 4, 'end': 7}]},
 {'id': 'SDRW2000000006.1.1.7',
  'form': '좋아하고',
  'word': [{'id': 1, 'form': '좋아하고', 'begin': 0, 'end': 4}],
  'NE': []},
 {'id': 'SDRW2000000006.1.1.8',
  'form': '카레도',
  'word': [{'id': 1, 'form': '카레도', 'begin': 0, 'end': 3}],
  'NE': [{'id': 1, 'form': '카레', 'label': 'CV_FOOD', 'begin': 0, 'end': 2}]},
 {'id': 'SDRW2000000006.1.1.9',
  'form': '매운 카레를 좋아하고',
  'word': [{'id': 1, 'form': '매운', 'begin': 0, 'end': 2},
   {'id': 2, 'form': '카레를', 'begin': 3, 'end': 6},
   {'id': 3, 'form': '좋아하고', 'begin': 7, 'end': 11}],
  'NE': [{'id': 1, 'form': '카레', 'label': 'CV_FOOD', 'begin': 3, 'end': 5}]},
 {'id': 'SDRW2000000006.1.1.10',
  'form': '돈가스를 먹어도',
  'word': [{'id': 1, 'form': '돈가스를', 'begin': 0, 'end': 4},
   {'id': 2, 'form': '먹어도', 'begi

In [8]:
from io import StringIO
import random

label_set = set(['UNK'])
sentences = []
labels = []

for document in test['document']:
    doc_sentences = document['sentence']

    sent = StringIO()
    label = []
    for words in doc_sentences:
        # sentence
        sent.write(words['form'].replace('|', ''))

        # label
        ne = words['NE']
        words_length = len(words['word'])
        if not ne:
            label.extend(['O'] * words_length)  # add O label
        else:
            ne_iter = iter(ne)
            next_label = next(ne_iter)
            try:
                for i, word in enumerate(words['word'], 1):
                    word_beg = word['begin']
                    word_end = word['end']

                    if next_label['begin'] <= word_end:  # reached label word
                        if word_beg <= next_label['begin']:  # if word start of label
                            label.append(f"{next_label['label']}-B")

                        else:  # label is multiple words, but does not contain begining
                            label.append(f"{next_label['label']}-I")

                        if next_label['end'] <= word_end:  # end of label
                            next_label = next(ne_iter)     
                    
                    else:
                        label.append('O')

            except StopIteration:
                label.extend(['O'] * (words_length - i))                

        # end of sentence
        if words['form'].endswith('.') or words['form'].endswith('?'):
            # separate . or ?
            # sent.seek(sent.tell()-1)
            # sent.write(f" {words['form'][-1]}")
            # label.append('O')

            label_set.update(label)

            sentences.append(sent.getvalue())
            labels.append(' '.join(label))
            sent = StringIO()  # new sentence
            label = []
            
        else:
            sent.write(' ')

with open('./data/train.tsv', 'w', encoding='utf-8') as f_train, open('./data/test.tsv', 'w', encoding='utf-8') as f_test:

    for s, l in zip(sentences, labels):
        rand = random.random()
        if rand < 0.8:
            f_train.write(f'{s}\t{l}\n')
        else:
            f_test.write(f'{s}\t{l}\n')

print(label_set)

{'QT_COUNT-B', 'OGG_SCIENCE-B', 'TR_SOCIAL_SCIENCE-I', 'FD_SCIENCE-I', 'CV_ART-B', 'OGG_FOOD-B', 'EV_FESTIVAL-I', 'AFA_PERFORMANCE-I', 'QT_ALBUM-B', 'OGG_SPORTS-I', 'PS_NAME-B', 'PT_TYPE-B', 'AF_CULTURAL_ASSET-I', 'AF_ROAD-I', 'QT_TEMPERATURE-I', 'PT_PART-B', 'CV_FOOD-I', 'PT_GRASS-B', 'UNK', 'OGG_ECONOMY-B', 'DT_DAY-I', 'CV_CULTURE-B', 'QT_PRICE-I', 'CV_OCCUPATION-I', 'OGG_LIBRARY-B', 'AM_FISH-B', 'CV_FOOD_STYLE-I', 'OGG_HOTEL-I', 'TMI_SERVICE-I', 'QT_WEIGHT-I', 'MT_ROCK-B', 'LC_OTHERS-B', 'EV_FESTIVAL-B', 'AM_PART-I', 'EV_SPORTS-I', 'OGG_MEDIA-B', 'TM_SHAPE-B', 'TR_SOCIAL_SCIENCE-B', 'AFW_SERVICE_PRODUCTS-I', 'FD_HUMANITIES-B', 'AFA_MUSIC-B', 'TMM_DISEASE-B', 'LCP_PROVINCE-B', 'EV_OTHERS-I', 'CV_POLICY-B', 'AF_BUILDING-I', 'DT_MONTH-I', 'AM_REPTILIA-B', 'PT_OTHERS-B', 'OGG_POLITICS-B', 'LCG_RIVER-B', 'TM_CLIMATE-B', 'OGG_POLITICS-I', 'FD_ART-I', 'EV_WAR_REVOLUTION-I', 'CV_CURRENCY-B', 'LCG_MOUNTAIN-B', 'TM_CELL_TISSUE_ORGAN-B', 'AFA_DOCUMENT-B', 'CV_DRINK-B', 'EV_ACTIVITY-B', 'DT_GEO

In [23]:
label_list = list(label_set)
label_list.sort()

with open('./data/label.txt', 'w', encoding='utf-8') as f_label:
    for lab in label_list:
        f_label.write(f'{lab}\n')


In [1]:
# test for cuda
import torch
torch.cuda.is_available()

True

In [ ]:
import easydict
from predict import pridict

pred_args = easydict.EasyDict({
    'input_file':'./sample_pred_in.txt',
    'output_file':'sample_pred_out.txt',
    'model_dir':'./model',
    'batch_size':32,
    'no_cuda':True,
 })

predict(pred_args)